## Assignment 3: Scraping the SEC Filings
#### Group 1: Tara Bode, Hankun Li, and Roberto Bettaglio

### Install Packages and Setup Environment

In [1]:
# Install Packages

!pip install w3lib first

import html, re
# need to do: pip install w3lib first
from w3lib.html import replace_entities

import csv
import os as os
import pandas as pd
import glob as glob

Defaulting to user installation because normal site-packages is not writeable


In [2]:
mydirectory = '/blue/acg7849/share/10Ks'
file_list = glob.glob(mydirectory + '/*.txt')
#print(file_list)

### Define Functions

In [3]:
def getHeaderVariables(filing):
    CIK = filing[ filing.find('CENTRAL INDEX KEY:') + 18  : filing.find('STANDARD') ]
    CIK = ''.join(CIK.split())
    #
    CCN = filing[ filing.find('COMPANY CONFORMED NAME:') + 23 : filing.find('CENTRAL INDEX KEY') ]
    CCN = CCN.rstrip().lstrip()
    #
    CST = filing[ filing.find('CONFORMED SUBMISSION TYPE:') + 26 : filing.find('PUBLIC DOCUMENT COUNT') ]
    CST = ''.join(CST.split())
    #
    CPR = filing[ filing.find('CONFORMED PERIOD OF REPORT:') + 27 : filing.find('FILED AS OF DATE') ]
    CPR = ''.join(CPR.split())
    #
    Length = 'skip'
    
    sec_info = '{} | {} | {} | {} | {}\n'.format(CIK, CCN, CST, CPR, Length)
    return (sec_info)

def cleanHTML(raw_html):
    cleanr = re.compile('<.*?>', flags=re.DOTALL)
    cleantext = re.sub(cleanr, '', raw_html)
    return replace_entities(cleantext)
    
def getBusinessSection(filing):
    # USE REGEXR
    BDS = filing[ filing.find('Item 1. ') : filing.find('Item 1A. ') ]
    #print('length_business_section',len(BDS))
    #print(BDS)
    return BDS
    #

### Writing the SEC Information to Main File

In [4]:
# write header variables and length of business section to disk
with open (r'/blue/acg7849/tbode/SEC_Information.csv', 'w') as f:
    f.write('File Name | Central Index Key | Company Name | Submission Type | Period of Report | Length of Business Description\n')

for file_path in file_list[0:1000]:
    # read the file
    with open(file_path) as f:
        filing = f.read()
        
    # only use until /DOCUMENT
    first_section = filing[0: filing.find('</DOCUMENT')]
    
    # get header variables
    headervars = str(getHeaderVariables(first_section))
    
    # isolate business section
    business_section = str(getBusinessSection(first_section))
    
    #need to input FileName
    shortName = file_path[file_path.rfind("/")+1:]
    
    # write business section to disk       
    with open(r'/blue/acg7849/tbode/item1/'+ shortName, 'w', encoding="utf-8") as f:
        f.write(cleanHTML(business_section))
   
    with open (r'/blue/acg7849/tbode/SEC_Information.csv', 'a', encoding="utf-8") as f:
        f.write(shortName + ' | ' + cleanHTML(headervars))

    #with open (r'/blue/acg7849/tbode/SEC_Information.csv', 'a', encoding="utf-8") as f:
     #   f.write(shortName + ' | ' + cleanHTML(headervars) + ' | ' + len(cleanHTML(business_section)))


### Previous Trial Runs for REGEX for Business Section that were Unsuccessful

In [ ]:
#business = '(?<=<a name="ITEM 1. DESCRIPTION OF BUSINESS.">)(.*?)(?=</a name="ITEM 1A. Risk Factors">)'
#business = '<[p style="MARGIN: 0px 0px 0px 0in" align="justify">&nbsp;</p> <p style="MARGIN: 0px 0px 0px 0in" align="justify">">]*>(.+?)<[</p> <p style="MARGIN: 0px 0px 0px 0in" align="justify">&nbsp;</p>]>'
#regexp = \{<p style="MARGIN: 0px 0px 0px 0in" align="justify">}(.*?)\{</p>}\m
#regexp = '<a name="ITEM 1. DESCRIPTION OF BUSINESS.">(.*?)<p style="MARGIN: 0px 0px 0px 0in" align="justify">'
#regexp = '<a name="PART 1">(.*?)<a name="ITEM 1A. Risk Factors">'
#regexp = '<b><a name="ITEM 1. DESCRIPTION OF BUSINESS.">(.*?)<a name="ITEM 1A. Risk Factors">'

line = file.read()

regexp = '(ITEM 1. DESCRIPTION OF BUSINESS.<)(.*?)(ITEM 1A. Risk Factors)'
# Clean newly extracted text
regexp = regexp.strip() # Remove starting/ending white spaces
regexp = regexp.replace('\n', ' ') # Replace \n (new line) with space
regexp = regexp.replace('\r', '') # Replace \r (carriage returns-if you're on windows) with space
regexp = regexp.replace(' ', ' ') # Replace " " (a special character for space in HTML) with space
regexp = regexp.replace(' ', ' ') # Replace " " (a special character for space in HTML) with space
while '  ' in regexp:
    regexp = regexp.replace('  ', ' ') # Remove extra spaces

# Print first 500 characters of newly extracted text
print(regexp[:50])

sig = re.findall(regexp, line)
sig
#sig.strip()



In [ ]:
# Business Section Regex Statement  ~ researched from https://gist.github.com/TobagoMango/7dd2dfe1e8174114f8f39fa8c5edcec8 ~
# wasn't able to run due to parsing issues - not sure how to adjust, as I tried different parsing combinations

import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define URL for the specific 10K filing
URL_text = glob.glob(mydirectory + '/*.txt')

# Grab the response
response = requests.get(URL_text)

# Parse the response
soup = BeautifulSoup(response.content, 'lxml')

for filing_document in soup.find_all('document'): 
    
    # The 'type' tag contains the document type
    document_type = filing_document.type.find(text=True, recursive=False).strip()
    
    if document_type == "10-K": # Once the 10K text body is found
        
        # Set up the regex pattern
        matches = re.compile(r'item\s(1[\.\s]|1a[\.\s])|(description\sof\sbusiness|business)|(risk\sfactors)', re.IGNORECASE)
    
        matches_array = pd.DataFrame([(match.group(), match.start()) for match in matches.finditer(filing)])
    
        # Set columns in the dataframe
        matches_array.columns = ['SearchTerm', 'Start']
        
        # Get the number of rows in the dataframe
        Rows = matches_array['SearchTerm'].count()
        print(Rows)
    
        # Create a new column in 'matches_array' called 'Selection' and add adjacent 'SearchTerm' (i and i+1 rows) text concatenated
        count = 0 # Counter to help with row location and iteration
        while count < (Rows-1): # Can only iterate to the second last row
            matches_array.at[count,'Selection'] = (matches_array.iloc[count,0] + matches_array.iloc[count+1,0]).lower() # Convert to lower case
            count += 1
        
        # Set up 'Item 1/1A Search Pattern' regex patterns
        matches_item1 = re.compile(r'(item\s1\.(description\sof\sbusiness|business)|[a-z]*)')
        matches_item1a = re.compile(r'(item\s1a\.(risk\sfactors)[a-z]*)')
            
        # Lists to store the locations of Item 1/1A Search Pattern matches
        Start_Loc = []
        End_Loc = []
    
        # Find and store the locations of Item 1/1A Search Pattern matches
        count = 0 # Set up counter
        
        while count < (Rows-1): # Can only iterate to the second last row
            
            # Match Item 1 Search Pattern
            if re.match(matches_item1, matches_array.at[count,'Selection']):
                # Column 1 = 'Start' columnn in 'matches_array'
                Start_Loc.append(matches_array.iloc[count,1]) # Store in list => Item 7 will be the starting location (column '1' = 'Start' column)
            
            # Match Item 1a Search Pattern
            if re.match(matches_item1a, matches_array.at[count,'Selection']):
                End_Loc.append(matches_array.iloc[count,1])
            
            count += 1
        
        # Extract section of text and store in 'TenKItem1'
        TenKItem1 = file_list[Start_Loc[1]:End_Loc[1]]
        
        # Clean newly extracted text
        TenKItem1 = TenKItem1.strip() # Remove starting/ending white spaces
        TenKItem1 = TenKItem1.replace('\n', ' ') # Replace \n (new line) with space
        TenKItem1 = TenKItem1.replace('\r', '') # Replace \r (carriage returns-if you're on windows) with space
        TenKItem1 = TenKItem1.replace(' ', ' ') # Replace " " (a special character for space in HTML) with space
        TenKItem1 = TenKItem1.replace(' ', ' ') # Replace " " (a special character for space in HTML) with space
        while '  ' in TenKItem1:
            TenKItem1 = TenKItem1.replace('  ', ' ') # Remove extra spaces

        # Print first 50 characters of extracted text
        print(TenKItem1[:50])